In [2]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import os
ROOT_DIR = './'

In [2]:
class MyntraScraper:
    def __init__(self):
        self.NUM_PAGES=6338
        self.headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
        
    def parseData(self, response):
        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script')

        for script in scripts:
            # print(script)
            if 'window.__myx = ' in script.text:
                json_str = script.text.split('window.__myx = ')[1]
                return json.loads(json_str)
            
    def saveData(self, page, scraped_json):
        with open(f'scraped/{page}.json', 'w') as f:
            json.dump(scraped_json, f)

    def parse_and_dump(page, response):
        scraped_json = parseData(response)
        saveData(page, scraped_json)
        
    def scrape_all(self):
        for page in range(1, self.NUM_PAGES + 1):
            print("Scraping: ", page)
            r = requests.get(f'https://www.myntra.com/womens-western-wear?p={page}', headers=self.headers)
            parse_and_dump(page, r)
            print("Saved:    ", page)
            time.sleep(0.4)
            
class MyntraProcessor:            
    def get_scraped_products(self):
        SELECTED_PRODUCT_KEYS = ['landingPageUrl', 'productId', 'product', 'productName', 'rating', 'ratingCount',
        'isFastFashion', 'brand', 'searchImage', 'sizes', 'gender', 'primaryColour', 'additionalInfo', 'category',
        'price', 'articleType', 'subCategory', 'masterCategory']

        all_products = []
        for file in os.listdir('scraped'):
            try:
                data = json.loads(open(f'scraped/{file}').read())
                products = data['searchData']['results']['products']
                filtered_product = [{key: product[key] for key in SELECTED_PRODUCT_KEYS} for product in products]
                all_products.extend(filtered_product)
            except Exception as e:
                print(f'Failed {file} : {e}')
        return all_products

    def write_to_csv(self, filename):
        scraped_products = get_scraped_products()
        pd.DataFrame(scraped_products).to_csv(filename)

In [ ]:
class InstagramScraper:
    def setup_dirs(self):
        if not os.path.exists(ROOT_DIR + 'scraped'):
            os.mkdir(ROOT_DIR + 'scraped')
        if not os.path.exists(ROOT_DIR + 'scraped/instagram'):
            os.mkdir(ROOT_DIR + 'scraped/instagram')
        
        if not os.path.exists(ROOT_DIR + 'related_usernames'):
            os.mkdir(ROOT_DIR + 'scraped/instagram/related_usernames')
        
    def __init__(self, usernames_to_scrape, max_usernames_to_scrape=1000):      
        self.scraped_usernames = set(os.listdir(ROOT_DIR + 'scraped/instagram'))
        self.usernames_to_scrape = []
        self.max_usernames_to_scrape = max_usernames_to_scrape
        
        self.headers ={
            # this is internal ID of an instegram backend app. It doesn't change often.
            "x-ig-app-id": "936619743392459",
            # use browser-like features
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.9,ru;q=0.8",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept": "*/*",
        }
        
        self.profile_url = 'https://i.instagram.com/api/v1/users/web_profile_info/?username={username}'

        
    ## Scrapes profile and saves it. 
    ## Returns usernames of related public profiles. 
    def scrape_profile(self, username):
        try:
            resp = requests.get(url.format(username=username), headers=self.headers, timeout=5)
            assert resp.status_code == 200, f'Failed to fetch {username} : {response.content}'
        except Exception as e:
            print(e)
            return []
        related_usernames = [x['node']['username'] for x in resp.json()['data']['user']['edge_related_profiles']['edges'] if not x['node']['is_private']]
        

In [3]:
set(os.listdir('scraped/instagram'))

FileNotFoundError: [Errno 2] No such file or directory: 'scraped/instagram'

In [12]:
print('hello: ', os.path.exists('scraped'))

hello:  True
